In [1]:
import requests
import csv
import re
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep

In [2]:
url = "https://www.mlssoccer.com/players/david-villa"
response = requests.get(url)
page = response.text

In [3]:
soup = BeautifulSoup(page, "html5lib")
name = (soup.find("title").text.split("|")[0])

In [4]:
stats = [["Player", "Year", "Club", "GP", "GS", "G", "MINS", "A", "SHTS", "SOG", "FC", "OFF", "Y", "R"]]
#game_stats = [["Player", "Date", "Match", "Result", "Appearance", "MINS", "G", "A", "SHTS", "SOG", "FC", "FS", "Y", "R"]]
name = (soup.find("title").text.split("|")[0])
pos = soup.find('span', {"class": "position"}).text
#year = True
row = [name]
for td in soup.find_all("td"):
    if (td["data-title"] == "Date"):
        break
    row.append(td.text)
    if (len(row) == 14):
        stats.append(row)
        row = [name]

In [5]:
url2 = "https://www.mlssoccer.com/rosters/2018"
response2 = requests.get(url2)
page2 = response2.text
soup2 = BeautifulSoup(page, "html5lib")

In [6]:
teams = soup2.optgroup
team_links = []
for option in teams:
    if (option["value"][-1] == "8"):
        pass
    else:
        team_links.append("https://www.mlssoccer.com/rosters/2018/" + option["value"].split("/")[-1])

In [7]:
url3 = "https://www.mlssoccer.com/rosters/2018/new-york-city-fc"
response3 = requests.get(url3)
page3 = response3.text
soup3 = BeautifulSoup(page3, "html5lib")

In [8]:
roster = soup3.table
rows = [row for row in roster.find_all('a')]
player_list = []
for row in rows:
    player_list.append("https://www.mlssoccer.com" + row["href"])
print (player_list)

['https://www.mlssoccer.com/players/Saad-Abdul-Salaam', 'https://www.mlssoccer.com/players/Kwame-Awuah', 'https://www.mlssoccer.com/players/Jo Inge-Berget', 'https://www.mlssoccer.com/players/Jeff-Caldwell', 'https://www.mlssoccer.com/players/Alexander-Callens', 'https://www.mlssoccer.com/players/Maxime-Chanot', 'https://www.mlssoccer.com/players/Yangel-Herrera', 'https://www.mlssoccer.com/players/Cedric-Hountondji', 'https://www.mlssoccer.com/players/Sebastien-Ibeagha', 'https://www.mlssoccer.com/players/Sean-Johnson', 'https://www.mlssoccer.com/players/Jonathan-Lewis', 'https://www.mlssoccer.com/players/Ronald-Matarrita', 'https://www.mlssoccer.com/players/Thomas-McNamara', 'https://www.mlssoccer.com/players/Jesus-Medina', 'https://www.mlssoccer.com/players/Maxi-Moralez', 'https://www.mlssoccer.com/players/Ebenezer-Ofori', 'https://www.mlssoccer.com/players/Andre-Rawls', 'https://www.mlssoccer.com/players/Alexander-Ring', 'https://www.mlssoccer.com/players/James-Sands', 'https://www.

In [9]:
# Certain URLs do not work.  Noting them here.
exclude_list = ["https://www.mlssoccer.com/players/Shaft-Brewer", "https://www.mlssoccer.com/players/Maximiano", "https://www.mlssoccer.com/players/Bertrand-Owundi-Eko",
               "https://www.mlssoccer.com/players/Zakaria-Diallo", "https://www.mlssoccer.com/players/Thomas-Meilleur-Giguere",
               "https://www.mlssoccer.com/players/Antonio-Delamea-Mlinar", "https://www.mlssoccer.com/players/Mark-Segbers",
               "https://www.mlssoccer.com/players/Ismael--Tajouri-Shradi", "https://www.mlssoccer.com/players/Earl-Edwards-Jr.",
               "https://www.mlssoccer.com/players/Amro-Tarek", "https://www.mlssoccer.com/players/Eryk-Williamson",
               "https://www.mlssoccer.com/players/Paul-Marie", "https://www.mlssoccer.com/players/Danny-Musovski",
               "https://www.mlssoccer.com/players/Mohamed-Thiaw", "https://www.mlssoccer.com/players/Aiden-Daniels",
               "https://www.mlssoccer.com/players/Simon-Colyn", "https://www.mlssoccer.com/players/Justin-Fiddes",
               "https://www.mlssoccer.com/players/David-Norman-Jr."]

In [10]:
stats = [["Player", "Position", "Year", "Club", "GP", "GS", "G", "MINS", "A", "SHTS", "SOG", "FC", "OFF", "Y", "R"]]
keepers = [["Player", "Position", "Year", "Club", "GP", "GS", "MINS", "SHO", "SV", "GA", "PKG", "PKA", "W", "L", "T"]]

for team in team_links:
    sleep(1)
    player_links = []
    response_team = requests.get(team)
    team_page = response_team.text
    team_soup = BeautifulSoup(team_page, "html5lib")
    roster = team_soup.table
    players = [temp for temp in roster.find_all('a')]
    player_links = []
    for player in players:
        player_links.append("https://www.mlssoccer.com" + player["href"])
        
    for link in player_links:
        link = re.sub(r'\s', '-', link)
        if (link in exclude_list):
            continue
        sleep(2)
        player_response = requests.get(link)
        player_page = player_response.text
        soup = BeautifulSoup(player_page, "html5lib")
        name = (soup.find("title").text.split("|")[0])
        pos = soup.find('span', {"class": "position"}).text
        row = [name, pos]
        for td in soup.find_all("td"):
            if (re.match("<td>.*?</td>", str(td))):
                continue
            if (td["data-title"] == "Date"):
                break
            row.append(td.text)
            if (len(row) == 15):
                if (pos == "Goalkeeper"):
                    keepers.append(row)
                else:
                    stats.append(row)
                row = [name, pos]

In [11]:
with open("output.csv", "w", newline = "") as out_file:
    writer = csv.writer(out_file)
    writer.writerows(stats)

with open("keepers.csv", "w", newline = "") as keep_outfile:
    writer_keep = csv.writer(keep_outfile)
    writer_keep.writerows(keepers)

In [12]:
print (td)
print (link)
print (team)

<td data-title="Date">3/04/2018</td>
https://www.mlssoccer.com/players/Marco-Bustos
https://www.mlssoccer.com/rosters/2018/vancouver-whitecaps
